<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PQLite-explained" data-toc-modified-id="PQLite-explained-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PQLite explained</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#How-does-pqlite-works" data-toc-modified-id="How-does-pqlite-works-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>How does pqlite works</a></span></li></ul></li></ul></li><li><span><a href="#Understanding-pq.fit" data-toc-modified-id="Understanding-pq.fit-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Understanding <code>pq.fit</code></a></span></li><li><span><a href="#Understanding-pq.add" data-toc-modified-id="Understanding-pq.add-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Understanding <code>pq.add</code></a></span></li><li><span><a href="#Understanding-pq.search" data-toc-modified-id="Understanding-pq.search-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Understanding <code>pq.search</code></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#How-does-pqlite-works" data-toc-modified-id="How-does-pqlite-works-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>How does pqlite works</a></span></li></ul></li><li><span><a href="#Time" data-toc-modified-id="Time-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Time</a></span></li><li><span><a href="#Quality" data-toc-modified-id="Quality-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Quality</a></span><ul class="toc-item"><li><span><a href="#Plotting-pq-neighbors-vs-true-neighbors" data-toc-modified-id="Plotting-pq-neighbors-vs-true-neighbors-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Plotting <code>pq neighbors</code> vs <code>true neighbors</code></a></span></li></ul></li><li><span><a href="#precision,-recall,-query_time-vs-n_subvectors-&amp;--n_cells" data-toc-modified-id="precision,-recall,-query_time-vs-n_subvectors-&amp;--n_cells-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span><code>precision, recall, query_time</code> vs <code>n_subvectors</code> &amp;  <code>n_cells</code></a></span></li></ul></li></ul></div>

## PQLite explained

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pyximport
pyximport.install()

import pqlite
pqlite.__path__
import time

import jina
from jina.math.distance import cdist

import sklearn
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

import random
import numpy as np
from pqlite import PQLite


#### How does pqlite works

Pqlite has a first coarse search step.

When adding elements to PQLite elements are stored in cells.

The number `n_datapoints / n_cells` will be roughly the number of elements in each cell.

In [3]:
Nq = 1
D = 128 
top_k = 100
n_cells = 10
n_subvectors = 32

## Understanding `pq.fit`

Internally, when doing `pq.fit(Xtr)` the `pq` class learns a quantizer stored in `pq.pq_codec`.

The `pq` does not add data unitl `pq.add()` is called.

We can see that the cells in `pq` are empty

Let us see what we have after adding to PQLIte with 500 examples


In [4]:

Nt = 500

np.random.seed(1234)
Xtr, Xte = train_test_split(make_blobs(n_samples = Nt, n_features = D)[0].astype(np.float32), test_size=20)

# the column schema: (name:str, dtype:type, create_index: bool)
pq = PQLite(d_vector=D, 
            n_cells=n_cells,
            n_subvectors=n_subvectors, 
            columns=[('x', float, True)])

pq.fit(Xtr)


2021-11-22 15:07:37.972 | INFO     | pqlite.index:fit:96 - => start training VQ codec with 480 data...
2021-11-22 15:07:37.990 | INFO     | pqlite.index:fit:99 - => start training PQ codec with 480 data...
2021-11-22 15:07:39.128 | INFO     | pqlite.index:fit:102 - => pqlite is successfully trained!


Information about hyperparams

In [5]:
pq.pq_codec.fit(Xtr)

In [6]:
print(f'pq.code_size = {pq.code_size}')
print(f'pq.d_subvector = {pq.d_subvector}')
print(f'pq.code_size = {pq.code_size}')
print(f'pq.n_subvectors = {pq.n_subvectors }')

pq.code_size = 32
pq.d_subvector = 4
pq.code_size = 32
pq.n_subvectors = 32


In [7]:
pq._cell_size

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Nevertheless we can use the current `quantizer` stored in `pq.pq_codec` to encode data 

In [8]:
pq.encode(Xte[[0]])

array([[132,  40,  25,  30, 160,  92,   9,  47, 131, 255,  17, 178,  41,
         53,   5,  52, 255, 153, 163,  80, 203, 164, 231, 106,   8,  98,
        243,  35, 201,  25,  74, 222]], dtype=uint8)

This quantizer uses a codebook for each of the subspaces in the Product space. 

Since we have code_size = 32 this means we will have 32 different subspaces, which will have been trained with the corresponding columns from the training data.

In this case since `pq.d_subvector` is 4 each of the slices `Xtr[:,0:4],Xtr[:,4:8],Xtr[:,8:12],....` will have a corresponding codebook. This matches because `128/32=4`.

All codebooks are stored in `pq.pq_codec.codebooks`

In [9]:
pq.pq_codec.codebooks.shape

(32, 256, 4)

And one of the codebooks contains a matrix of shape `(K,d_subvector)` where `K` is the number of prototypes for each subspace. 

In [10]:
pq.pq_codec.codebooks[0].shape

(256, 4)

##### Understanding the encoding

Once we have fitted a `pq.codec` we can encode the data.
This process takes a real valued vector, splits it in slices of size `pq.d_subvector` and each of the slices is assigned to the closest prototype stored in the codebook of the corresponding slice.

For example we can take an slice of a vector and look where it should be matched


In [11]:
slice_0 = Xte[0][0:4]
slice_0

array([-5.496039 ,  6.284065 ,  1.7528343,  2.411145 ], dtype=float32)

In [12]:
dists_to_prototypes_slice_0 = np.sum((pq.pq_codec.codebooks[0] - slice_0)**2, axis=1)
print(dists_to_prototypes_slice_0.shape)
print(dists_to_prototypes_slice_0.argmin())

(256,)
132


Repeating this process for each slice will encode our vector in the PQ space.

This can be done using `pq.encode`

In [13]:
pq.encode(Xte[[0]])

array([[132,  40,  25,  30, 160,  92,   9,  47, 131, 255,  17, 178,  41,
         53,   5,  52, 255, 153, 163,  80, 203, 164, 231, 106,   8,  98,
        243,  35, 201,  25,  74, 222]], dtype=uint8)

This method will internally call the stored `pq_codec` and call the `.encode` method of the internal `pq_codec`

In [14]:
pq.pq_codec.encode(Xte[[0]])

array([[132,  40,  25,  30, 160,  92,   9,  47, 131, 255,  17, 178,  41,
         53,   5,  52, 255, 153, 163,  80, 203, 164, 231, 106,   8,  98,
        243,  35, 201,  25,  74, 222]], dtype=uint8)

##### pq._vecs_storage

pq stores the quantized data in `pq._vecs_storage`. This is a list of `n_cells` elements containing matrices with the quantized data added to `pq`. Note that if no data is added this matrices will contain only 0 values.


In [15]:
len(pq._vecs_storage) == pq.n_cells

True

In [16]:
len(pq._vecs_storage) 

10

If we look inside `_vecs_storage[k]` we will see that everything is zero because we have not added data yet

In [17]:
pq._vecs_storage[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

This matrices will start with shape `pq.expand_step_size` and will grow once cells allocate all the possible rows

In [18]:
print(f'pq.expand_step_size = {pq.expand_step_size}')
print(f'pq._vecs_storage[0].shape = {pq._vecs_storage[0].shape}')

pq.expand_step_size = 1024
pq._vecs_storage[0].shape = (1024, 32)


Note that matrices have as many columns as `pq.n_subvectors`. 

A vector is partitioned into subvectors each of which is represented as a single integer.

In [19]:
print(f'pq._vecs_storage[0].shape[1] = {pq._vecs_storage[0].shape[1]}')
print(f'pq.n_subvectors = {pq.n_subvectors}')

pq._vecs_storage[0].shape[1] = 32
pq.n_subvectors = 32


## Understanding `pq.add`

We have seen that `pq` has not stored a single example

In [20]:
pq._cell_size

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

To add examples we have to do 

In [21]:
pq.add(Xtr, ids=list(range(len(Xtr))))

2021-11-22 15:07:43.974 | DEBUG    | pqlite.storage.cell:_expand:148 - => total storage capacity is expanded by 0 for 10 cells
2021-11-22 15:07:43.975 | DEBUG    | pqlite.storage.cell:insert:90 - => 480 new items added


Now can see that each example is added to a cell. 

In [22]:
pq._cell_size

array([62, 72, 76, 38, 41, 78, 29, 50,  6, 28])

The cell information is 

In [23]:
print(f'The number of cells is n_cells={pq.n_cells}')

print('\nCells can be accessed in pq.cell_tables')
print(f'\twe have len(pq.cell_tables)={len(pq.cell_tables)} cells')
print(f'\nWe have added len(Xtr)={len(Xtr)} elements to pq')

The number of cells is n_cells=10

Cells can be accessed in pq.cell_tables
	we have len(pq.cell_tables)=10 cells

We have added len(Xtr)=480 elements to pq


Note that `pq.cell_tables` is a list of  `CellTable` objects

In [24]:
pq.cell_tables[0]


Each CellTable allows you to `insert`, `query` and `delete` vectors

We can inspect how many elements are in a cell using `.count()`

In [25]:
pq.cell_tables[0].count()

62

Not all `cell_tables` will contain the same number of elements because not all of them are assigned to the same prototype. Nevertheless the sum of the elements across cells equalts the number of added elements

In [26]:
elements_per_cell = [pq_cell_table.count() for pq_cell_table in pq.cell_tables]
print('elements_per_cell =', elements_per_cell)
print('total number of elements added =', np.sum(elements_per_cell))
print('np.sum(elements_per_cell) == len(Xtr) is ',np.sum(elements_per_cell) == len(Xtr))

elements_per_cell = [62, 72, 76, 38, 41, 78, 29, 50, 6, 28]
total number of elements added = 480
np.sum(elements_per_cell) == len(Xtr) is  True


## Understanding `pq.search`


Internally `pq.search` first computes the distance between each query and the prototypes that define the cells. Then the cells whose prototypes are closest to a query are selected as search space. The best  `pq.n_probe` cells are selected (this is a hyperparameter of the algorithm).

Since `pq.n_probe` in this case is bigger than `pq.n_cells` all the cells will be searched.

In [27]:
pq.n_probe, pq.n_cells

(16, 10)

Note that  `pq.search` can be called with a batch of vectors. Once called it will end up calling `pq.search_cells` with the full batch of queries but with an array of arrays containing at each position a list of the ids of the cells that best batch the query. So if 5 queries are passed into the `pq.search` it will pass to `self.search_cells` an array of size `(len(queries), max(pq.n_probe, pq.n_cells)`.

The `.search_cells` method iterates over the queries and comptues the distance between each query and all retrieved elements in the activated cells.

For each query in the batch  the Asymetric Distance Computation is performed using `pq.pq_codec.precompute_adc` which returns a table of shape `(pq.n_subvectors, pq.pq_codec.n_clusters)`.  In our case a matrix of shape `(32, 256)`.


In [28]:
query = Xtr[[10]]

In [29]:
dtable = pq.pq_codec.precompute_adc(query[0])
dtable.dtable.shape

(32, 256)

We can do this faster with a cython function as follows

In [30]:
from pqlite.utils.asymmetric_distance import precompute_adc_table

In [31]:
dt = precompute_adc_table(query[0], 
                          pq.d_subvector,
                          pq.pq_codec.n_clusters,
                          pq.pq_codec.codebooks)

In [32]:
np.mean(np.asarray(dt) - dtable.dtable)

-3.0230967e-07

In [33]:
%%timeit 
precompute_adc_table(query[0], 
                     pq.d_subvector,
                     pq.pq_codec.n_clusters,
                     pq.pq_codec.codebooks)

50.4 µs ± 820 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
%%timeit 
pq.pq_codec.precompute_adc(query[0])

458 µs ± 6.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We can make sure that both tables are almost equal

In [35]:
np_dtable_1 = np.asarray(precompute_adc_table(query[0], 
                         pq.d_subvector,
                         pq.pq_codec.n_clusters,
                         pq.pq_codec.codebooks))

np_dtable_2 = pq.pq_codec.precompute_adc(query[0]).dtable

np.testing.assert_array_almost_equal(np_dtable_1, np_dtable_2,decimal=4)
np.testing.assert_array_almost_equal_nulp(np_dtable_1, np_dtable_2)

This table contains the distance between each possible subvector in que query and each possible subvector from any subcodevector.

Therefore we go from `search` -> `search_cells` -> `search_cells` -> `precomputed_k = pq_codec.precompute_adc(query_k)` -> `ivfpq_topk`

Therefore for `query_k` we compute the ADC table. Then this table is used to compute the distance between the query and all the database. 

In this case, since there is filtering, the computations are done only on a subset of the database. Distances are computed between the query and the exapmles that come from the selected cells and verify the conditions specified by the provided filter.

```
self.ivfpq_topk(precomputed, cells=cell_idx,conditions=conditions,k=k )
```

In [36]:
pq.pq_codec.codebooks.shape

(32, 256, 4)

In [37]:
precomputed = pq.pq_codec.precompute_adc(query[0])
precomputed.dtable.shape

(32, 256)

Given a bunch of datapoint candidates from the database (from which we already have the pqcode) we want to find distances between the query and the candidates. This is done with `precomputed.adist(codes)` which returns the distance between each code in codes and the pqcode of the query.

In [38]:
pq.pq_codec.d_subvector, pq.pq_codec.n_clusters

(4, 256)

In [39]:
indices = list(range(200))
codes = pq.vecs_storage[0][indices]

In [ ]:

Nt = 500

np.random.seed(1234)
Xtr, Xte = train_test_split(make_blobs(n_samples = Nt, n_features = D)[0].astype(np.float32), test_size=20)

# the column schema: (name:str, dtype:type, create_index: bool)
pq = PQLite(d_vector=D, 
            n_cells=n_cells,
            n_subvectors=n_subvectors, 
            columns=[('x', float, True)])

pq.fit(Xtr)

In [ ]:
Nq = 1
query = Xtr[[10,4]] 
pq_dists, ids = pq.search(query, k=5)

Then the best `k` matches are returned as well as the distances to those matches.

Note that, because distances are computed using the reconstruction values of the quantized vectors an expected distance of 0  in the original space will not be zero.

For example here we would expect the distance between the query and the best match to be 0 because it has been added to pq, but this is not the case because of the quantization error.

The search process for a given query is based on computing the distance between the query and all datapoints. This is done precomputing the distance between the query (all n_subvectors from the query) and all the subcodevectors of the Product space.

In [ ]:
adc_for_query = pq.pq_codec.precompute_adc(Xtr[10])
adc_for_query.dtable.shape

In [ ]:
#pqlite.core.codec.pq.DistanceTable()

In [ ]:
pq.pq_codec.codebooks.shape

In [ ]:
Nq = 1
query = Xtr[[10]] 
pq_dists, ids = pq.search(query, k=5)

print(pq_dists)
print(ids)

In [ ]:
print(pq.pq_codec.codebooks.shape)
print((pq.n_subvectors, pq.pq_codec.n_clusters, pq.d_subvector))

In [ ]:
codes

#### How does pqlite works

Pqlite has a first coarse search step.

When adding elements to PQLite elements are stored in cells.


In [ ]:
#current search
%timeit pq_dists, ids = pq.search(query, k=100)

In [ ]:
from pqlite.utils import asymmetric_distance

In [ ]:
asymmetric_distance.dist_pqcodes_to_codebooks

In [ ]:
query_pqcodes = pq.encode(query)

In [ ]:
query_pqcodes

In [ ]:
query_pqcodes_int32 = query_pqcodes.astype('int32')

In [ ]:
print(pq.pq_codec.codebooks.shape)
M = pq.pq_codec.codebooks.shape[0]

In [ ]:
pq.pq_codec.codebooks.shape[0:2]

In [ ]:
cell_0_pq_codebook = pq.vecs_storage[0]

In [ ]:
len(pq.vecs_storage)

In [ ]:
dtable = pq.pq_codec.precompute_adc(query[0]).dtable
print(dtable.shape)
pqlite.utils.asymmetric_distance.dist_pqcodes_to_codebooks(M, 
                                                           dtable,
                                                           query_pqcodes_int32)

### Time

In [ ]:
%timeit  pq.search(query,  k=top_k)

In [ ]:
from pqlite.utils import asymmetric_distance

In [ ]:
asymmetric_distance.dist_pqcode_to_codebooks

In [ ]:
%timeit res = cdist(query, Xtr, metric='euclidean')

### Quality

Manually observing slices of a high dimensional space it seems that the
retrieved items from pqlite are nearby the query and the best values from the exact and exhaustive distance computations.

In [ ]:
def _precision(predicted, relevant, eval_at):
    """
    fraction of retrieved documents that are relevant to the query
    """
    if eval_at == 0:
        return 0.0
    predicted_at_k = predicted[:eval_at]
    n_predicted_and_relevant = len(set(predicted_at_k).intersection(set(relevant)))

    return n_predicted_and_relevant / len(predicted)

def _recall(predicted, relevant, eval_at):
    """
    fraction of the relevant documents that are successfully retrieved
    """
    if eval_at == 0:
        return 0.0
    predicted_at_k = predicted[:eval_at]
    n_predicted_and_relevant = len(set(predicted_at_k).intersection(set(relevant)))
    return n_predicted_and_relevant/ len(relevant)


In [ ]:
query = Xte[[11]]  
true_distances = cdist(query, Xtr, metric='euclidean').flatten()

true_ids = np.argsort(true_distances)[0:top_k]
true_dists = true_distances[true_ids]

In [ ]:
true_ids.sort()
true_ids

In [ ]:
pq_dists, pq_ids = pq.search(query,  k=top_k)
pq_ids = np.array([int(x) for x in pq_ids[0]])

In [ ]:
pq_ids.sort()
pq_ids

In [ ]:
print(_precision(true_ids, pq_ids, top_k))
print(_recall(true_ids, pq_ids, top_k))

#### Plotting `pq neighbors` vs `true neighbors`

In [ ]:
import matplotlib.pyplot as plt

def paint_slice(Xtr, query, feat1, feat2):
    plt.scatter(Xtr[:,feat1], Xtr[:,feat2], color='blue', alpha=0.2)

    for pq_id in pq_ids:
        plt.scatter(Xtr[pq_id, feat1], Xtr[pq_id, feat2], color='black')

    for true_id in true_ids:
        plt.scatter(Xtr[true_id, feat1], Xtr[true_id, feat2], color='orange')

    plt.scatter(query[:, feat1], query[:, feat2], color='red')
    

In [ ]:
feat1, feat2 = 0, 1
paint_slice(Xtr, query, feat1, feat2)

In [ ]:
feat1, feat2 = 8, 100
paint_slice(Xtr, query, feat1, feat2)

In [ ]:
feat1, feat2 = 3,4
paint_slice(Xtr, query, feat1, feat2)


###  `precision, recall, query_time` vs `n_subvectors` &  `n_cells` 

In [ ]:
import time
import numpy as np
from pqlite import PQLite

from jina.math.distance import cdist
from jina.math.helper import top_k as _top_k
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

def _precision(predicted, relevant, eval_at):
    """
    fraction of retrieved documents that are relevant to the query
    """
    if eval_at == 0:
        return 0.0
    predicted_at_k = predicted[:eval_at]
    n_predicted_and_relevant = len(set(predicted_at_k).intersection(set(relevant)))

    return n_predicted_and_relevant / len(predicted)

def _recall(predicted, relevant, eval_at):
    """
    fraction of the relevant documents that are successfully retrieved
    """
    if eval_at == 0:
        return 0.0
    predicted_at_k = predicted[:eval_at]
    n_predicted_and_relevant = len(set(predicted_at_k).intersection(set(relevant)))
    return n_predicted_and_relevant/ len(relevant)

def evaluate(predicts, relevants, eval_at):
    recall = 0
    precision = 0
    for _predict, _relevant in zip(predicts, relevants):
        _predict = np.array([int(x) for x in _predict])
        recall += _recall(_predict, _relevant, top_k)
        precision += _precision(_predict, _relevant, top_k)

    return recall / len(predicts), precision / len(predicts)


#N = 100_000 # number of data points
Nt = 125_000
Nq = 1
D = 128 # dimentionality / number of features
top_k = 10
n_cells = 64
n_subvectors = 64
n_queries = 1000

# 2,000 128-dim vectors for training
np.random.seed(123)
Xtr, Xte = train_test_split(make_blobs(n_samples = Nt, n_features = D)[0].astype(np.float32), test_size=20)
print(f'Xtr: {Xtr.shape} vs Xte: {Xte.shape}')

precision_per_query = []
recall_per_query = []
results = []

for n_cells in [8, 16, 32, 64, 128]:
    for n_subvectors in [32, 64, 128]:

        pq = PQLite(d_vector=D,
                    n_cells=n_cells,
                    n_subvectors=n_subvectors)

        t0 = time.time()
        pq.fit(Xtr[:20480])
        train_time = abs(time.time() - t0)

        t0 = time.time()
        pq.add(Xtr, ids=list(range(len(Xtr))))
        index_time = abs(t0 - time.time())

        dists = cdist(Xte, Xtr, metric='euclidean')
        true_dists, true_ids = _top_k(dists, top_k, descending=False)

        t0 = time.time()
        pq_dists, pq_ids = pq.search(Xte, k=top_k)
        query_time = abs(t0 - time.time())

        recall, precision = evaluate(pq_ids, true_ids, top_k)

        results_dict = {'precision': precision,
                        'recall': recall,
                        'train_time': train_time,
                        'index_time': index_time,
                        'query_time': query_time,
                        'indexer_hyperparams': {'n_cells': n_cells,
                                                'n_subvectors': n_subvectors}
                        }
        print(results_dict)

        results.append(results_dict)

In [ ]:
import pandas as pd
results_df = pd.DataFrame(results)
results_df.sort_values('recall', ascending=False)